In [ ]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 477.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 5.7 MB/s eta 0:00:00


In [ ]:
import langgraph.graph as lg

print(dir(lg))

['END', 'MessageGraph', 'MessagesState', 'START', 'StateGraph', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_branch', '_node', 'add_messages', 'message', 'state']


In [ ]:
[name for name in dir(lg) if not name.startswith("_")]

['END',
 'MessageGraph',
 'MessagesState',
 'START',
 'StateGraph',
 'add_messages',
 'message',
 'state']

In [ ]:
from langgraph.graph import StateGraph, MessageGraph, MessagesState

print(dir(StateGraph))
print(dir(MessageGraph))
print(dir(MessagesState))


['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_schema', '_all_edges', 'add_conditional_edges', 'add_edge', 'add_node', 'add_sequence', 'compile', 'set_conditional_entry_point', 'set_entry_point', 'set_finish_point', 'validate']
['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__deprecated__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr_

In [ ]:
import inspect

def list_methods(cls):
    return [name for name, obj in inspect.getmembers(cls, inspect.isfunction) if not name.startswith("_")]

print("StateGraph:", list_methods(StateGraph))
print("MessageGraph:", list_methods(MessageGraph))
print("MessagesState:", list_methods(MessagesState))


StateGraph: ['add_conditional_edges', 'add_edge', 'add_node', 'add_sequence', 'compile', 'set_conditional_entry_point', 'set_entry_point', 'set_finish_point', 'validate']
MessageGraph: ['add_conditional_edges', 'add_edge', 'add_node', 'add_sequence', 'compile', 'set_conditional_entry_point', 'set_entry_point', 'set_finish_point', 'validate']
MessagesState: []


In [ ]:
import importlib
import inspect
import dataclasses
import typing as t

# --- Optional schema detection support ---
try:
    from pydantic import BaseModel as _PydanticBaseModel
except Exception:
    _PydanticBaseModel = None

try:
    import typing
    _TypedDictMeta = typing.TypedDict.__class__
except Exception:
    _TypedDictMeta = None

# ------------------ helpers ------------------

def _is_public(name: str) -> bool:
    return not name.startswith("_")

def _is_function(obj: t.Any) -> bool:
    return inspect.isfunction(obj) or inspect.isbuiltin(obj)

def _is_class(obj: t.Any) -> bool:
    return inspect.isclass(obj)

def _is_module(obj: t.Any) -> bool:
    return inspect.ismodule(obj)

def _looks_like_typed_dict(cls: t.Any) -> bool:
    return _TypedDictMeta and isinstance(cls, _TypedDictMeta)

def _looks_like_pydantic_model(cls: t.Any) -> bool:
    return _PydanticBaseModel and inspect.isclass(cls) and issubclass(cls, _PydanticBaseModel)

def _looks_like_dataclass(cls: t.Any) -> bool:
    return dataclasses.is_dataclass(cls)

def _class_public_methods(cls: type) -> list[str]:
    methods = []
    for name, member in inspect.getmembers(cls):
        if not _is_public(name):
            continue
        if inspect.isfunction(member) or inspect.ismethoddescriptor(member):
            methods.append(name)
        elif isinstance(member, (classmethod, staticmethod)):
            methods.append(name)
    return sorted(set(methods))

def _class_public_attributes(cls: type) -> list[str]:
    attrs = []
    for name, member in inspect.getmembers(cls):
        if not _is_public(name):
            continue
        if _is_module(member) or _is_function(member):
            continue
        if inspect.ismethod(member) or inspect.ismethoddescriptor(member):
            continue
        attrs.append(name)
    return sorted(set(attrs))

def _short_type(obj: t.Any) -> str:
    return type(obj).__name__

def _safe_repr(obj: t.Any, maxlen: int = 80) -> str:
    try:
        r = repr(obj)
    except Exception:
        r = f"<{_short_type(obj)}>"
    if len(r) > maxlen:
        r = r[: maxlen - 1] + "…"
    return r

# ------------------ main API ------------------

def explore_module(module_name: str) -> dict:
    mod = importlib.import_module(module_name)
    classes, functions, constants = {}, [], []

    for name in dir(mod):
        if not _is_public(name):
            continue
        obj = getattr(mod, name)
        if _is_module(obj):
            continue
        if _is_class(obj):
            if _looks_like_pydantic_model(obj):
                schema_kind = "pydantic"
            elif _looks_like_dataclass(obj):
                schema_kind = "dataclass"
            elif _looks_like_typed_dict(obj):
                schema_kind = "typeddict"
            else:
                schema_kind = None
            classes[name] = {
                "schema_kind": schema_kind,
                "methods": _class_public_methods(obj),
                "attributes": _class_public_attributes(obj),
            }
        elif _is_function(obj):
            functions.append(name)
        else:
            constants.append(
                {"name": name, "type": _short_type(obj), "repr": _safe_repr(obj)}
            )

    return {
        "module": module_name,
        "classes": dict(sorted(classes.items())),
        "functions": sorted(functions),
        "constants": sorted(constants, key=lambda d: d["name"]),
    }

def pretty_print_report(report: dict) -> None:
    mod = report["module"]
    print(f"\n=== Module: {mod} ===")

    print("\nClasses:")
    if not report["classes"]:
        print("  (none)")
    else:
        for cls_name, meta in report["classes"].items():
            schema = f" [schema={meta['schema_kind']}]" if meta["schema_kind"] else ""
            print(f"  - {cls_name}{schema}")
            print(f"      methods: {meta['methods']}")
            print(f"      attributes: {meta['attributes']}")

    print("\nFunctions:")
    if not report["functions"]:
        print("  (none)")
    else:
        for fn in report["functions"]:
            print(f"  - {fn}")

    print("\nConstants / Objects / Sentinels:")
    if not report["constants"]:
        print("  (none)")
    else:
        for c in report["constants"]:
            print(f"  - {c['name']} : <{c['type']}> = {c['repr']}")

# ------------------ Example usage in notebook ------------------

# Example: explore langgraph.graph
report = explore_module("langgraph.graph")
pretty_print_report(report)


=== Module: langgraph.graph ===

Classes:
  - MessageGraph
      methods: ['add_conditional_edges', 'add_edge', 'add_node', 'add_sequence', 'compile', 'set_conditional_entry_point', 'set_entry_point', 'set_finish_point', 'validate']
      attributes: []
  - MessagesState
      methods: ['clear', 'copy', 'get', 'items', 'keys', 'pop', 'popitem', 'setdefault', 'update', 'values']
      attributes: []
  - StateGraph
      methods: ['add_conditional_edges', 'add_edge', 'add_node', 'add_sequence', 'compile', 'set_conditional_entry_point', 'set_entry_point', 'set_finish_point', 'validate']
      attributes: []

Functions:
  - add_messages

Constants / Objects / Sentinels:
  - END : <str> = '__end__'
  - START : <str> = '__start__'


In [ ]:
# === Notebook-friendly module & package explorer with colored pretty print ===
from __future__ import annotations

import importlib
import inspect
import dataclasses
import typing as t
import pkgutil

# ---------- Optional schema detection support ----------
try:
    from pydantic import BaseModel as _PydanticBaseModel
except Exception:
    _PydanticBaseModel = None

try:
    import typing
    _TypedDictMeta = typing.TypedDict.__class__
except Exception:
    _TypedDictMeta = None


# ---------- ANSI color helpers (safe if colors unsupported) ----------
class _C:
    BOLD = "\033[1m"
    DIM = "\033[2m"
    RESET = "\033[0m"
    BLUE = "\033[34m"
    CYAN = "\033[36m"
    GREEN = "\033[32m"
    MAGENTA = "\033[35m"
    YELLOW = "\033[33m"
    GRAY = "\033[90m"

def _c(s: str, color: str) -> str:
    return f"{color}{s}{_C.RESET}"


# ---------- Introspection helpers ----------
def _is_public(name: str) -> bool:
    return not name.startswith("_")

def _is_function(obj: t.Any) -> bool:
    return inspect.isfunction(obj) or inspect.isbuiltin(obj)

def _is_class(obj: t.Any) -> bool:
    return inspect.isclass(obj)

def _is_module(obj: t.Any) -> bool:
    return inspect.ismodule(obj)

def _looks_like_typed_dict(cls: t.Any) -> bool:
    return _TypedDictMeta and isinstance(cls, _TypedDictMeta)

def _looks_like_pydantic_model(cls: t.Any) -> bool:
    return _PydanticBaseModel and inspect.isclass(cls) and issubclass(cls, _PydanticBaseModel)

def _looks_like_dataclass(cls: t.Any) -> bool:
    return dataclasses.is_dataclass(cls)

def _class_public_methods(cls: type) -> list[str]:
    methods = []
    for name, member in inspect.getmembers(cls):
        if not _is_public(name):
            continue
        if inspect.isfunction(member) or inspect.ismethoddescriptor(member):
            methods.append(name)
        elif isinstance(member, (classmethod, staticmethod)):
            methods.append(name)
    return sorted(set(methods))

def _class_public_attributes(cls: type) -> list[str]:
    attrs = []
    for name, member in inspect.getmembers(cls):
        if not _is_public(name):
            continue
        if _is_module(member) or _is_function(member):
            continue
        if inspect.ismethod(member) or inspect.ismethoddescriptor(member):
            continue
        attrs.append(name)
    return sorted(set(attrs))

def _short_type(obj: t.Any) -> str:
    try:
        return type(obj).__name__
    except Exception:
        return "<unknown>"

def _safe_repr(obj: t.Any, maxlen: int = 80) -> str:
    try:
        r = repr(obj)
    except Exception:
        r = f"<{_short_type(obj)}>"
    if len(r) > maxlen:
        r = r[: maxlen - 1] + "…"
    return r


# ---------- Module explorer ----------
def explore_module(module_name: str) -> dict:
    """
    Import a module and return a structured dictionary:
      - classes (with schema_kind, methods, attributes)
      - functions
      - constants/objects/sentinels
    """
    mod = importlib.import_module(module_name)
    classes: dict[str, dict] = {}
    functions: list[str] = []
    constants: list[dict[str, str]] = []

    for name in dir(mod):
        if not _is_public(name):
            continue
        obj = getattr(mod, name)
        if _is_module(obj):
            continue

        if _is_class(obj):
            if _looks_like_pydantic_model(obj):
                schema_kind = "pydantic"
            elif _looks_like_dataclass(obj):
                schema_kind = "dataclass"
            elif _looks_like_typed_dict(obj):
                schema_kind = "typeddict"
            else:
                schema_kind = None
            classes[name] = {
                "schema_kind": schema_kind,
                "methods": _class_public_methods(obj),
                "attributes": _class_public_attributes(obj),
            }
        elif _is_function(obj):
            functions.append(name)
        else:
            constants.append(
                {"name": name, "type": _short_type(obj), "repr": _safe_repr(obj)}
            )

    return {
        "module": module_name,
        "classes": dict(sorted(classes.items(), key=lambda kv: kv[0].lower())),
        "functions": sorted(functions, key=str.lower),
        "constants": sorted(constants, key=lambda d: d["name"].lower()),
    }


# ---------- Pretty printer ----------
def pretty_print_report(report: dict) -> None:
    mod = report["module"]
    print(_c(f"\n=== Module: {mod} ===", _C.BOLD + _C.BLUE))

    print(_c("\nClasses:", _C.BOLD + _C.MAGENTA))
    if not report["classes"]:
        print("  (none)")
    else:
        for cls_name, meta in report["classes"].items():
            schema = f" [schema={meta['schema_kind']}]" if meta["schema_kind"] else ""
            print(_c(f"  - {cls_name}{schema}", _C.GREEN))
            methods = meta["methods"]
            attrs = meta["attributes"]
            print(_c("      methods:", _C.DIM), methods if methods else "[]")
            print(_c("      attributes:", _C.DIM), attrs if attrs else "[]")

    print(_c("\nFunctions:", _C.BOLD + _C.MAGENTA))
    if not report["functions"]:
        print("  (none)")
    else:
        for fn in report["functions"]:
            print(_c("  - ", _C.GREEN) + fn)

    print(_c("\nConstants / Objects / Sentinels:", _C.BOLD + _C.MAGENTA))
    if not report["constants"]:
        print("  (none)")
    else:
        for c in report["constants"]:
            name = _c(c["name"], _C.GREEN)
            ctype = _c(c["type"], _C.CYAN)
            rep = _c(c["repr"], _C.GRAY)
            print(f"  - {name} : <{ctype}> = {rep}")


# ---------- Package (library) explorer ----------
def list_modules(package_name: str, recursive: bool = True, include_private: bool = False) -> list[str]:
    """
    Return a list of importable modules/submodules under a package.

    Args:
        package_name: top-level package (e.g. "langgraph")
        recursive: walk subpackages recursively
        include_private: if False, skip dotted-path parts that start with "_"
    """
    pkg = importlib.import_module(package_name)

    # If it's a plain module, just return itself
    if not hasattr(pkg, "__path__"):
        return [package_name]

    results: list[str] = []
    for _, name, _ in pkgutil.walk_packages(pkg.__path__, prefix=f"{package_name}."):
        if not include_private:
            parts = name.split(".")
            if any(p.startswith("_") for p in parts):
                continue
        results.append(name)
    return sorted(results)


def explore_package(
    package_name: str,
    recursive: bool = True,
    include_private: bool = False,
    limit: int | None = None,
) -> None:
    """
    List modules in a package, then print a per-module report.

    Args:
        package_name: e.g. "langgraph"
        recursive: recurse into subpackages
        include_private: include modules whose path parts start with "_"
        limit: if set, limit how many modules to report (useful for very large packages)
    """
    modules = list_modules(package_name, recursive=recursive, include_private=include_private)

    print(_c(f"\n=== Modules in '{package_name}' ({'including' if include_private else 'excluding'} private) ===",
             _C.BOLD + _C.BLUE))
    for m in modules if (limit is None) else modules[:limit]:
        print("  -", m)

    print(_c(f"\n=== Detailed reports ===", _C.BOLD + _C.BLUE))
    targets = modules if (limit is None) else modules[:limit]
    for m in targets:
        try:
            report = explore_module(m)
            pretty_print_report(report)
        except Exception as e:
            print(_c(f"\n=== Module: {m} ===", _C.BOLD + _C.BLUE))
            print(_c("  (failed to import) ", _C.YELLOW) + repr(e))


# ------------------ Example usage in notebook ------------------
# First: list modules, then show a report for each (public only)
explore_package("langgraph", recursive=True, include_private=False)
# If the package is huge, you can limit output:
# explore_package("langgraph", recursive=True, include_private=False, limit=10)



=== Modules in 'langgraph' (excluding private) ===
  - langgraph.channels
  - langgraph.channels.any_value
  - langgraph.channels.base
  - langgraph.channels.binop
  - langgraph.channels.ephemeral_value
  - langgraph.channels.last_value
  - langgraph.channels.named_barrier_value
  - langgraph.channels.topic
  - langgraph.channels.untracked_value
  - langgraph.config
  - langgraph.constants
  - langgraph.errors
  - langgraph.func
  - langgraph.graph
  - langgraph.graph.message
  - langgraph.graph.state
  - langgraph.graph.ui
  - langgraph.managed
  - langgraph.managed.base
  - langgraph.managed.is_last_step
  - langgraph.prebuilt
  - langgraph.prebuilt.chat_agent_executor
  - langgraph.prebuilt.interrupt
  - langgraph.prebuilt.tool_node
  - langgraph.prebuilt.tool_validator
  - langgraph.pregel
  - langgraph.pregel.debug
  - langgraph.pregel.main
  - langgraph.pregel.protocol
  - langgraph.pregel.remote
  - langgraph.pregel.types
  - langgraph.runtime
  - langgraph.types
  - langgraph.

In [ ]:
import inspect
from langgraph.graph import StateGraph
print(inspect.signature(StateGraph.__init__))

(self, state_schema: 'type[StateT]', context_schema: 'type[ContextT] | None' = None, *, input_schema: 'type[InputT] | None' = None, output_schema: 'type[OutputT] | None' = None, **kwargs: 'Unpack[DeprecatedKwargs]') -> 'None'


In [ ]:
# === Auto-detect "class(class)" constructor params in a module ===
from __future__ import annotations
import importlib, inspect, typing as t
from typing import get_origin, get_args, get_type_hints

# Heuristic names that usually indicate a class/type param
_CLASSY_NAMES = {
    "cls", "klass", "class_", "impl", "implementation",
    "factory", "adapter", "strategy",
    "serializer_cls", "state_schema", "schema",
    "model", "model_cls", "type_", "node_type", "backend_cls"
}

def _is_public(name: str) -> bool:
    return not name.startswith("_")

def _public_classes_in_module(mod) -> dict[str, type]:
    out = {}
    for name in dir(mod):
        if not _is_public(name):
            continue
        obj = getattr(mod, name)
        if inspect.isclass(obj):
            out[name] = obj
    return out

def _annotation_wants_class(ann) -> bool:
    """True if annotation looks like type[T] / typing.Type[T] / raw type."""
    try:
        origin = get_origin(ann)
        if origin is type:  # e.g. type[T]
            return True
        txt = str(ann)
        if "typing.Type[" in txt or "type[" in txt:
            return True
    except Exception:
        pass
    return ann is type  # raw 'type'

def _annotation_base_type(ann):
    """Return T from type[T]/Type[T], else None."""
    try:
        origin = get_origin(ann)
        if origin is type:
            args = get_args(ann)
            if args and inspect.isclass(args[0]):
                return args[0]
    except Exception:
        pass
    return None

def _name_suggests_class(param_name: str) -> bool:
    if param_name in _CLASSY_NAMES:
        return True
    if param_name.endswith("_class") or param_name.endswith("_cls"):
        return True
    return False

def find_class_params(module_name: str) -> list[dict]:
    """
    For each public class in module, analyze __init__ params and
    guess which ones expect a CLASS. Suggest candidate classes.
    """
    mod = importlib.import_module(module_name)
    pub_classes = _public_classes_in_module(mod)

    reports = []
    for cls_name, cls in sorted(pub_classes.items()):
        try:
            sig = inspect.signature(cls.__init__)
        except Exception:
            continue

        # evaluated hints work best; fall back to raw annotations if needed
        try:
            hints = get_type_hints(cls.__init__, globalns=vars(mod), include_extras=True)
        except Exception:
            hints = {}

        params_info = []
        for pname, p in sig.parameters.items():
            if pname == "self":
                continue

            ann = hints.get(pname, p.annotation)
            wants_class = _annotation_wants_class(ann) or _name_suggests_class(pname)

            if wants_class:
                base = _annotation_base_type(ann)
                candidates = []
                if base and inspect.isclass(base):
                    # suggest classes in this module that are subclasses of the base
                    for cand_name, cand_cls in pub_classes.items():
                        try:
                            if issubclass(cand_cls, base) and cand_cls is not base:
                                candidates.append(cand_name)
                        except Exception:
                            pass
                else:
                    # unknown base -> soft suggestions: all public classes in this module
                    candidates = [n for n in pub_classes.keys() if n != cls_name]

                params_info.append({
                    "param": pname,
                    "annotation": str(ann),
                    "base_type": base.__name__ if isinstance(base, type) else None,
                    "candidates": sorted(candidates)
                })

        if params_info:
            reports.append({
                "class": f"{module_name}.{cls_name}",
                "init_signature": str(sig),
                "expects_class_params": params_info
            })

    return reports

def print_class_param_report(reports: list[dict]) -> None:
    if not reports:
        print("No constructor parameters that clearly expect a CLASS were found.")
        return
    for rep in reports:
        print(f"\n=== {rep['class']} ===")
        print(f"__init__ signature: {rep['init_signature']}")
        for p in rep["expects_class_params"]:
            print(f"  - param: {p['param']}")
            print(f"      annotation: {p['annotation']}")
            if p["base_type"]:
                print(f"      expects: CLASS (subclass of {p['base_type']})")
            else:
                print(f"      expects: CLASS (base type unclear; check docs/examples)")
            print(f"      candidate classes in module: {p['candidates']}")

# ---------------- Example: run on langgraph.graph ----------------
reports = find_class_params("langgraph.graph")
print_class_param_report(reports)



=== langgraph.graph.StateGraph ===
__init__ signature: (self, state_schema: 'type[StateT]', context_schema: 'type[ContextT] | None' = None, *, input_schema: 'type[InputT] | None' = None, output_schema: 'type[OutputT] | None' = None, **kwargs: 'Unpack[DeprecatedKwargs]') -> 'None'
  - param: state_schema
      annotation: type[StateT]
      expects: CLASS (base type unclear; check docs/examples)
      candidate classes in module: ['MessageGraph', 'MessagesState']
  - param: context_schema
      annotation: type[ContextT] | None
      expects: CLASS (base type unclear; check docs/examples)
      candidate classes in module: ['MessageGraph', 'MessagesState']
  - param: input_schema
      annotation: type[InputT] | None
      expects: CLASS (base type unclear; check docs/examples)
      candidate classes in module: ['MessageGraph', 'MessagesState']
  - param: output_schema
      annotation: type[OutputT] | None
      expects: CLASS (base type unclear; check docs/examples)
      candidate 

In [ ]:
from __future__ import annotations
import importlib, inspect, typing as t
from typing import get_origin, get_args, get_type_hints

def _is_public(name: str) -> bool:
    return not name.startswith("_")

def _public_classes(mod) -> dict[str, type]:
    out = {}
    for name in dir(mod):
        if _is_public(name):
            obj = getattr(mod, name)
            if inspect.isclass(obj):
                out[name] = obj
    return out

def _ann_wants_class(ann) -> bool:
    try:
        origin = get_origin(ann)
        if origin is type:           # e.g. type[T]
            return True
        txt = str(ann)
        if "typing.Type[" in txt or "type[" in txt:
            return True
    except Exception:
        pass
    return ann is type              # raw 'type'

def _ann_base_type(ann):
    try:
        if get_origin(ann) is type:
            args = get_args(ann)
            if args and inspect.isclass(args[0]):
                return args[0]
    except Exception:
        pass
    return None

def list_class_ctor_type_params(module_name: str) -> list[dict]:
    mod = importlib.import_module(module_name)
    classes = _public_classes(mod)
    reports = []

    for cls_name, cls in sorted(classes.items()):
        try:
            sig = inspect.signature(cls.__init__)
        except Exception:
            continue
        try:
            hints = get_type_hints(cls.__init__, globalns=vars(mod), include_extras=True)
        except Exception:
            hints = {}

        params = []
        for pname, p in sig.parameters.items():
            if pname == "self":
                continue
            ann = hints.get(pname, p.annotation)
            if _ann_wants_class(ann):
                base = _ann_base_type(ann)
                # suggest candidate classes from same module if base known
                candidates = []
                if base:
                    for cand_name, cand_cls in classes.items():
                        try:
                            if issubclass(cand_cls, base) and cand_cls is not base:
                                candidates.append(cand_name)
                        except Exception:
                            pass
                params.append({
                    "param": pname,
                    "annotation": str(ann),
                    "base_type": base.__name__ if isinstance(base, type) else None,
                    "candidates_in_module": sorted(candidates) if candidates else None
                })

        if params:
            reports.append({
                "class": f"{module_name}.{cls_name}",
                "init_signature": str(sig),
                "type_params": params
            })
    return reports

def pretty_print_type_param_report(module_name: str):
    reps = list_class_ctor_type_params(module_name)
    if not reps:
        print(f"(No classes in {module_name} with constructor params typed as type[T]/Type[T].)")
        return
    for r in reps:
        print(f"\n=== {r['class']} ===")
        print(f"__init__: {r['init_signature']}")
        for p in r["type_params"]:
            print(f"  - param: {p['param']}")
            print(f"      annotation: {p['annotation']}")
            print(f"      expects: CLASS (type)")
            if p['base_type']:
                print(f"      base type: {p['base_type']}")
            if p['candidates_in_module']:
                print(f"      candidates in module: {p['candidates_in_module']}")

# Example: run for langgraph.graph
pretty_print_type_param_report("langgraph.graph")


=== langgraph.graph.StateGraph ===
__init__: (self, state_schema: 'type[StateT]', context_schema: 'type[ContextT] | None' = None, *, input_schema: 'type[InputT] | None' = None, output_schema: 'type[OutputT] | None' = None, **kwargs: 'Unpack[DeprecatedKwargs]') -> 'None'
  - param: state_schema
      annotation: type[StateT]
      expects: CLASS (type)
  - param: context_schema
      annotation: type[ContextT] | None
      expects: CLASS (type)
  - param: input_schema
      annotation: type[InputT] | None
      expects: CLASS (type)
  - param: output_schema
      annotation: type[OutputT] | None
      expects: CLASS (type)


In [ ]:
# scan the whole package (public modules) for constructor params that expect classes
mods = list_modules("langgraph", recursive=True, include_private=False)

for m in mods:
    reps = list_class_ctor_type_params(m)  # from the helper we added earlier
    if not reps:
        continue
    print(f"\n### Module: {m}")
    for r in reps:
        print(f"  - {r['class']}")
        print(f"    __init__: {r['init_signature']}")
        for p in r["type_params"]:
            print(f"      param: {p['param']} | {p['annotation']}  → expects CLASS")
            if p["base_type"]:
                print(f"        base type: {p['base_type']}")
            if p["candidates_in_module"]:
                print(f"        candidates: {p['candidates_in_module']}")



### Module: langgraph.channels
  - langgraph.channels.BinaryOperatorAggregate
    __init__: (self, typ: type[~Value], operator: Callable[[~Value, ~Value], ~Value])
      param: typ | type[~Value]  → expects CLASS
  - langgraph.channels.NamedBarrierValue
    __init__: (self, typ: type[~Value], names: set[~Value]) -> None
      param: typ | type[~Value]  → expects CLASS
  - langgraph.channels.NamedBarrierValueAfterFinish
    __init__: (self, typ: type[~Value], names: set[~Value]) -> None
      param: typ | type[~Value]  → expects CLASS
  - langgraph.channels.Topic
    __init__: (self, typ: 'type[Value]', accumulate: 'bool' = False) -> 'None'
      param: typ | type[Value]  → expects CLASS
  - langgraph.channels.UntrackedValue
    __init__: (self, typ: 'type[Value]', guard: 'bool' = True) -> 'None'
      param: typ | type[Value]  → expects CLASS

### Module: langgraph.channels.binop
  - langgraph.channels.binop.BinaryOperatorAggregate
    __init__: (self, typ: type[~Value], operator: Cal

In [ ]:
# === Generate "ClassA(ClassB)" combos per module for any package ===
from __future__ import annotations
import importlib, inspect, pkgutil, typing as typing
from typing import get_origin, get_args, get_type_hints

# ---------- module listing ----------
def list_modules(package_name: str, recursive: bool = True, include_private: bool = False) -> list[str]:
    pkg = importlib.import_module(package_name)
    if not hasattr(pkg, "__path__"):
        return [package_name]
    out = []
    for _, name, _ in pkgutil.walk_packages(pkg.__path__, prefix=f"{package_name}."):
        if not include_private and any(part.startswith("_") for part in name.split(".")):
            continue
        out.append(name)
    return sorted(out)

# ---------- helpers ----------
SCHEMAY_NAMES = {"schema","state_schema","model","model_cls","cls","class_","*_cls","*_class"}
VALUE_TYPE_NAMES = {"typ","type","value_type"}

def _is_public(name: str) -> bool:
    return not name.startswith("_")

def _public_classes(mod) -> dict[str, type]:
    out = {}
    for n in dir(mod):
        if not _is_public(n):
            continue
        obj = getattr(mod, n)
        if inspect.isclass(obj):
            out[n] = obj
    return out

def _ann_wants_class(ann) -> bool:
    try:
        origin = get_origin(ann)
        if origin is type:
            return True
        s = str(ann)
        if "typing.Type[" in s or "type[" in s:
            return True
    except Exception:
        pass
    return ann is type

def _ann_base_type(ann):
    try:
        if get_origin(ann) is type:
            args = get_args(ann)
            if args and inspect.isclass(args[0]):
                return args[0]
    except Exception:
        pass
    return None

def _name_is_schemaish(pname: str) -> bool:
    if pname in SCHEMAY_NAMES:
        return True
    if pname.endswith("_schema") or pname.endswith("_cls") or pname.endswith("_class"):
        return True
    return False

def _name_is_value_typeish(pname: str) -> bool:
    return pname in VALUE_TYPE_NAMES

# ---------- core: build combos ----------
def combos_for_module(module_name: str) -> list[str]:
    """
    Return one-liner combos like 'StateGraph(MessagesState)' for a given module.
    """
    mod = importlib.import_module(module_name)
    classes = _public_classes(mod)
    combos: list[str] = []

    for cls_name, cls in sorted(classes.items()):
        # get __init__ signature & hints

        if "deprecated" in cls_name.lower():
          continue
        try:
            sig = inspect.signature(cls.__init__)
        except Exception:
            continue
        try:
            hints = get_type_hints(cls.__init__, globalns=vars(mod), include_extras=True)
        except Exception:
            hints = {}

        for pname, param in sig.parameters.items():
            if pname == "self":
                continue
            ann = hints.get(pname, param.annotation)
            if not _ann_wants_class(ann):
                continue

            # If base type is concrete, suggest subclasses in same module
            base = _ann_base_type(ann)
            if base and inspect.isclass(base):
                candidates = []
                for other_name, other_cls in classes.items():
                    try:
                        if other_cls is not base and issubclass(other_cls, base):
                            candidates.append(other_name)
                    except Exception:
                        pass
                for cand in sorted(set(candidates)):
                    combos.append(f"{cls_name}({cand})")
                # If we found concrete candidates, continue; otherwise fall through to heuristics
                if candidates:
                    continue

            # Heuristic 1: schema-like param names -> prefer classes with State/Model/Schema in name
            if _name_is_schemaish(pname):
                schemaish = [n for n in classes.keys() if any(tok in n.lower() for tok in ("state","model","schema")) and n != cls_name]
                for cand in sorted(set(schemaish)):
                    combos.append(f"{cls_name}({cand})")
                if schemaish:
                    continue

            # Heuristic 2: value-type params ('typ'/'type') -> suggest builtins
            if _name_is_value_typeish(pname):
                for builtin in ("int","str","dict"):
                    combos.append(f"{cls_name}({builtin})")
                continue

            # Fallback: if we got here, we still know it's a class; offer all other classes in module
            for cand in sorted(n for n in classes.keys() if n != cls_name):
                combos.append(f"{cls_name}({cand})")

    # Deduplicate while preserving order
    seen = set()
    uniq = []
    for c in combos:
        if c not in seen:
            seen.add(c)
            uniq.append(c)
    return uniq

def combos_for_package(package_name: str, recursive: bool = True, include_private: bool = False):
    """
    Print combos per module in a package.
    """
    mods = list_modules(package_name, recursive=recursive, include_private=include_private)
    print(f"=== Modules in '{package_name}' ===")
    for m in mods:
        print("  -", m)
    for m in mods:
        lines = combos_for_module(m)
        if not lines:
            continue
        print(f"\n### {m}")
        for line in lines:
            print(" -", line)

# --------- Example: run for langgraph, public modules only ---------
# Only this call is needed:
combos_for_package("langgraph", recursive=True, include_private=False)

# If you only want a single module:
# for line in combos_for_module("langgraph.graph"):
#     print(line)


=== Modules in 'langgraph' ===
  - langgraph.channels
  - langgraph.channels.any_value
  - langgraph.channels.base
  - langgraph.channels.binop
  - langgraph.channels.ephemeral_value
  - langgraph.channels.last_value
  - langgraph.channels.named_barrier_value
  - langgraph.channels.topic
  - langgraph.channels.untracked_value
  - langgraph.config
  - langgraph.constants
  - langgraph.errors
  - langgraph.func
  - langgraph.graph
  - langgraph.graph.message
  - langgraph.graph.state
  - langgraph.graph.ui
  - langgraph.managed
  - langgraph.managed.base
  - langgraph.managed.is_last_step
  - langgraph.prebuilt
  - langgraph.prebuilt.chat_agent_executor
  - langgraph.prebuilt.interrupt
  - langgraph.prebuilt.tool_node
  - langgraph.prebuilt.tool_validator
  - langgraph.pregel
  - langgraph.pregel.debug
  - langgraph.pregel.main
  - langgraph.pregel.protocol
  - langgraph.pregel.remote
  - langgraph.pregel.types
  - langgraph.runtime
  - langgraph.types
  - langgraph.typing
  - langgraph.